In [5]:

import numpy as np

from entities.entity import Entity
from entities.animal import Animal 
from entities.microphone import MicrophoneStation

SPEED_OF_SOUND = 343 # m/s

In [6]:
# the following function generated using GPT-4
def trilaterate(p1, r1, p2, r2, p3, r3):
    # Calculate relative positions of point 2 and point 3
    ex = (p2 - p1) / np.linalg.norm(p2 - p1)
    i = np.dot(ex, p3 - p1)
    ey = (p3 - p1 - i * ex) / np.linalg.norm(p3 - p1 - i * ex)
    ez = np.cross(ex, ey)

    # Calculate the distances
    d = np.linalg.norm(p2 - p1)
    j = np.dot(ey, p3 - p1)

    # Calculate the position of the intersection point
    x = (r1 ** 2 - r2 ** 2 + d ** 2) / (2 * d)
    y = (r1 ** 2 - r3 ** 2 + i ** 2 + j ** 2) / (2 * j) - (i / j) * x

    # Calculate the z-coordinate, if possible
    z_squared = r1 ** 2 - x ** 2 - y ** 2
    if z_squared < 0:
        return None  # No intersection
    z = np.sqrt(z_squared)

    # Calculate the coordinates
    p = p1 + x * ex + y * ey + z * ez
    return p

In [7]:
# Set an animal true location
truth_animal = Animal()
truth_animal.setLLA((-48.0,134.0,10.0))
print(f'True animal LLA', truth_animal.getLLA())

# set the location of three sensors around this animal
sensor_1 = MicrophoneStation(lla=(-48.0,133.99,10.0))
print(f'Sensor 1    LLA', sensor_1.getLLA())

sensor_2 = MicrophoneStation(lla=(-48.0,134.05,10.0))
print(f'Sensor 2    LLA', sensor_2.getLLA())

sensor_3 = MicrophoneStation(lla=(-48.01,134.0,10.0))
print(f'Sensor 3    LLA', sensor_3.getLLA())

print("distance 1", sensor_1.distance(truth_animal))
print("distance 2", sensor_2.distance(truth_animal))
print("distance 3", sensor_3.distance(truth_animal))

# Sphere centers and radii
p1 = np.array(sensor_1.getECEF())
r1 = sensor_1.distance(truth_animal)
sound_travel_time_1 = r1 / SPEED_OF_SOUND

p2 = np.array(sensor_2.getECEF())
r2 = sensor_2.distance(truth_animal)
sound_travel_time_2 = r2 / SPEED_OF_SOUND

p3 = np.array(sensor_3.getECEF())
r3 = sensor_3.distance(truth_animal)
sound_travel_time_3 = r3 / SPEED_OF_SOUND

print("sound travel time 1", sound_travel_time_1)
print("sound travel time 2", sound_travel_time_2)
print("sound travel time 3", sound_travel_time_3)

# Trilateration
intersection = trilaterate(p1, r1, p2, r2, p3, r3)
if intersection is not None:
    print(f"Intersection point ECEF: {intersection}")
    predicted_animal = Animal()
    predicted_animal.setECEF(intersection)
    print(f'Predicted animal LLA', predicted_animal.getLLA())
else:
    print("No intersection found.")

True animal LLA (-48.0, 134.0, 10.0)
Sensor 1    LLA (-48.0, 133.99, 10.0)
Sensor 2    LLA (-48.0, 134.05, 10.0)
Sensor 3    LLA (-48.01, 134.0, 10.0)
distance 1 746.2547025302385
distance 2 3731.2733989899125
distance 3 1111.9059424847765
sound travel time 1 2.17566968667708
sound travel time 2 10.878348102011406
sound travel time 3 3.2417082871276284
Intersection point ECEF: [-2970167.60763061  3075698.59488686 -4716883.43772499]
Predicted animal LLA (-48.00000000042609, 133.99999999863581, 9.5641894293949)


In [12]:
# grid search
estimate_animal = Animal()
best_err = 100000000

for lat in np.arange(-48.1,-47.9,0.01):
    for lon in np.arange(133.9,134.1,0.01):
        
        # guess it is here...
        estimate_animal.setLLA((lat, lon, 10.0))
        
        # so calculate the times given that guess
        p1 = np.array(sensor_1.getECEF())
        r1 = sensor_1.distance(estimate_animal)
        s_1 = r1 / SPEED_OF_SOUND

        p2 = np.array(sensor_2.getECEF())
        r2 = sensor_2.distance(estimate_animal)
        s_2 = r2 / SPEED_OF_SOUND

        p3 = np.array(sensor_3.getECEF())
        r3 = sensor_3.distance(estimate_animal)
        s_3 = r3 / SPEED_OF_SOUND
        
        # calculate the error with times observed
        err = abs(s_1-sound_travel_time_1) + \
              abs(s_2-sound_travel_time_2) + \
              abs(s_3-sound_travel_time_3) 
              
        if err < best_err:
            best_err = err
            print(f'New best lat: {lat} lon: {lon} absolute error in arrival times: {err}')
        
        

New best lat: -48.1 lon: 133.9 absolute error in arrival times: 103.92429126440295
New best lat: -48.1 lon: 133.91 absolute error in arrival times: 100.07840720269687
New best lat: -48.1 lon: 133.92 absolute error in arrival times: 96.48161776965028
New best lat: -48.1 lon: 133.92999999999998 absolute error in arrival times: 93.15806569673114
New best lat: -48.1 lon: 133.93999999999997 absolute error in arrival times: 90.13269464713886
New best lat: -48.1 lon: 133.94999999999996 absolute error in arrival times: 87.43065625164112
New best lat: -48.1 lon: 133.95999999999995 absolute error in arrival times: 85.0765521498733
New best lat: -48.1 lon: 133.96999999999994 absolute error in arrival times: 83.09353841057948
New best lat: -48.1 lon: 133.97999999999993 absolute error in arrival times: 81.50234972821046
New best lat: -48.1 lon: 133.98999999999992 absolute error in arrival times: 80.32032873481067
New best lat: -48.1 lon: 133.99999999999991 absolute error in arrival times: 79.560562